# Mixing Experiments for Kikuchi Patterns

Quick, interactive sandbox for exploring **synthetic mixing** strategies that map directly to the dataset generator (`scripts/generate_data.py`).


## How to use

- Run the next cell to open an interactive UI (pipeline, weights, preprocessing, blur/noise).
- Use it to decide sensible settings, then copy them into `configs/default.yaml` (or a custom config) for dataset generation.


In [ ]:
from pathlib import Path
import sys

# Repo path setup (so `src/` imports work)
ROOT = Path("..").resolve()
if not (ROOT / "src").exists():
    ROOT = Path(".").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.utils.mixing_experiments import MixingExperimentConfig, show_interactive_mixer

# Default: real experimental BCC/FCC reference patterns (change these as needed).
# The UI will populate A/B choices from `A_PATH.parent`.
GOOD_PATTERN_DIR = ROOT / "data" / "raw" / "Double Pattern Data" / "Good Pattern"
A_PATH = GOOD_PATTERN_DIR / "Perfect_BCC-1.bmp"
B_PATH = GOOD_PATTERN_DIR / "Perfect_FCC-1.bmp"

cfg = MixingExperimentConfig(a_path=A_PATH, b_path=B_PATH)
show_interactive_mixer(cfg)
